# CSCI 3155 Spring 2023
# Recitation Week 4
This week we will be reviewing case definition, pattern matching and introducing a new concept, inference rules.

# Exercise 1

We have defined a grammar for arithmetic expressions in the lecture. 

$$\begin{array}{rcl}
\mathbf{Expr} & \Rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{String}) \\ 
& | & Plus (\mathbf{Expr}, \mathbf{Expr}) \\
& | & Minus (\mathbf{Expr}, \mathbf{Expr}) \\
& | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Exp(\mathbf{Expr}) \\
& | & Sin(\mathbf{Expr}) \\
& | & Cos(\mathbf{Expr}) \\
\end{array}$$

In [1]:
sealed trait Expr

// Const-rule
case class Const(v: Double) extends Expr
case class Ident(x: String) extends Expr

// Binary-op-rule
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr

// Unary-op-rule
case class Exp(e: Expr) extends Expr
case class Sin(e: Expr) extends Expr
case class Cos(e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Exp
defined class Sin
defined class Cos

In [2]:
val c = Const(1)
val x = Ident("x")

val e1 = Plus(Const(1), Const(2))
val e2 = Minus(Ident("x"), Ident("y"))
val e3 = Mult(Ident("x"), Const(1))
val e4 = Div(Const(1), Ident("x"))

val e5 = Exp(e1)
val e6 = Sin(e2)
val e7 = Cos(e3)

c: Const = Const(v = 1.0)
x: Ident = Ident(x = "x")
e1: Plus = Plus(e1 = Const(v = 1.0), e2 = Const(v = 2.0))
e2: Minus = Minus(e1 = Ident(x = "x"), e2 = Ident(x = "y"))
e3: Mult = Mult(e1 = Ident(x = "x"), e2 = Const(v = 1.0))
e4: Div = Div(e1 = Const(v = 1.0), e2 = Ident(x = "x"))
e5: Exp = Exp(e = Plus(e1 = Const(v = 1.0), e2 = Const(v = 2.0)))
e6: Sin = Sin(e = Minus(e1 = Ident(x = "x"), e2 = Ident(x = "y")))
e7: Cos = Cos(e = Mult(e1 = Ident(x = "x"), e2 = Const(v = 1.0)))

# Review inference rules

In this recitation, we will focus on inference rules.


Inference rules are a logic form that takes premises and returns conclusions. They generally have the standard form of the following:

$$\begin{array}{c}
preconditions\\
\hline
postconditions\\
\end{array}\ (\text{Rule Name}) \leftarrow
\begin{array}{c}
\text{ Must check if these conditions satisfy } \\
\hline
\text{ Return postconditions } \\
\end{array}
$$

You can read them as "if all the preconditions are satisfied, then the postconditions."

# Example: Check if an expression is "well-formed"

We call an expression ***"well-formed"*** if all variables that appear in it are defind before being used.

The inference rules tell us how to implement a function that checks well-formedness. As implementors, we read the rules backwards. For example, $\text{(binary-op-rule)}$ tells us that, in order to check well-formedness of the expression `Mult(e1, e2)` given a sequence $l$ of defined variables, we need to
1. Check if `e1` is well-formed
2. Check if `e2` is well-formed under a sequence of defined variables $x :: l$

$\fbox{WellFormed(e, l) = b}$

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, l) = true \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
(x \text{ is in } l) = b\\
\hline
WellFormed(\texttt{Ident(x)}, l) = b\\
\end{array} \text{(ident-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, l)=b_1 \;\;\; WellFormed(\texttt{e2}, l)=b_2\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Div}\} \\
\hline
WellFormed(\texttt{T(e1, e2)}, l) = b_1 \ \&\&\  b_2\\
\end{array} \text{(binary-op-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, l)=b \;\;\; T \in \{\texttt{Exp}, \texttt{Sin}, \texttt{Cos}\} \\
\hline
WellFormed(\texttt{T(e1)}, l) = b \\
\end{array} \text{(unary-op-rule)} $$

In [3]:
def isWellFormed(e: Expr, seq: List[String]): Boolean = {
    
    def isWellFormed_unaryOp(e: Expr) = {
        isWellFormed(e, seq)
    }

    def isWellFormed_binOp(e1: Expr, e2: Expr) = {
        // mutual recursion, see https://en.wikipedia.org/wiki/Mutual_recursion
        // preconditions && preconditions == postconditions
        isWellFormed(e1, seq) && isWellFormed(e2, seq)
    }

    e match {
        // const-rule        
        case Const(_)  => true                  // evaluate preconditions and return postconditions
        case Ident(x) => seq.contains(x)        // evaluate preconditions and return postconditions
        
        // unary operations
        case Exp(e) => isWellFormed_unaryOp(e)
        case Sin(e) => isWellFormed_unaryOp(e)
        case Cos(e) => isWellFormed_unaryOp(e)

        // binary operations
        case Plus(e1, e2) => isWellFormed_binOp(e1, e2)
        case Minus(e1, e2) => isWellFormed_binOp(e1, e2)
        case Mult(e1, e2) => isWellFormed_binOp(e1, e2)
        case Div(e1, e2) => isWellFormed_binOp(e1, e2)
    }
}


defined function isWellFormed

In [4]:
assert(isWellFormed(Const(1.0), List()), "Test 1")
assert(isWellFormed(Plus(Const(1.0), Ident("z")), List("z")), "Test 2")
assert(isWellFormed(Plus(Sin(Const(0.5)), Const(2.9)), List()), "Test 3")
assert(!isWellFormed(Plus(Sin(Const(0.5)), Ident("x")), List()), "Test 4")

# Excercise 2: Implement a derivative function

#### A few Differentiation INFERENCE Rules

A rule for CONSTANTS :

$$\frac{dc}{dx} = 0, c \in \mathbb{R}$$

$$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Const(f)} , x) = \texttt{Const(0.0)} \\
\end{array} \mathbf{(Constant)}$$      

A rule for IDENTIFIERS:

$$\frac{dx}{dx} = 1, \frac{dy}{dx} = 0 \text{ for } y \not= x$$

$$\begin{array}{c}
\\
\hline 
\text{derivative}( \texttt{Ident(s)} , x) = \left\{ \begin{array}{ll} \texttt{Const(1.0)} & x == s \\
\texttt{Const(0.0)} & \text{otherwise} \end{array} \right.\\
\end{array} \mathbf{(Identifier)}  \;\;\;
$$

A rule for ADDITION :

$$\frac{d}{dx} (e_1 + e_2) = \frac{de_1}{dx} + \frac{de_2}{dx}$$

$$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Plus(e1, e2)}, x) = \texttt{Plus(f1, f2)} \\
\end{array} \mathbf{(Plus)} $$

A rule for MULTIPLICATION: 

$$\frac{d}{dx} (e_1 e_2) = e_2 \frac{de_1}{dx} + e_1 \frac{de_2}{dx}$$

$$ \begin{array}{c}
\text{derivative}(\texttt{e1}, x) = \texttt{f1},\;\;\text{derivative}(\texttt{e2}, x) = \texttt{f2}\\
\hline
\text{derivative}(\texttt{Mult(e1, e2)}, x) = \texttt{Plus(Mult(f1, e2), Mult(f2, e1))} \\
\end{array} \mathbf{(Mult)} $$


In [5]:
// A Function: derivative
def derivative(e: Expr, x: String): Expr =
    e match  {
        case Const(_) => Const(0)
        case Ident(s) => if (s == x) { Const(1.0)} else {Const(0.0)}
        // BEGIN SOLUTION
        case Plus(e1, e2) => Plus(derivative(e1, x), derivative(e2, x))
        case Mult(e1, e2) => Plus( Mult(derivative(e1, x), e2), Mult(derivative(e2, x), e1))
        // END SOLUTION
    }

defined function derivative

In [6]:
val InputExpression = Plus(Const(50), Ident("x"))
val OutputExpression = derivative(InputExpression, "x")

assert(OutputExpression == Plus(Const(0), Const(1)), "Test")
println("Input Expr: " + InputExpression)
println("Output Expr: " + OutputExpression)

Input Expr: Plus(Const(50.0),Ident(x))
Output Expr: Plus(Const(0.0),Const(1.0))


InputExpression: Plus = Plus(e1 = Const(v = 50.0), e2 = Ident(x = "x"))
OutputExpression: Expr = Plus(e1 = Const(v = 0.0), e2 = Const(v = 1.0))

In [7]:
val InputExpression = Mult(Const(50), Ident("x"))
val OutputExpression = derivative(InputExpression, "x")

assert(OutputExpression == Plus(Mult(Const(0), Ident("x")), Mult(Const(1), Const(50))), "Test")
println("Input Expr: " + InputExpression)
println("Output Expr: " + OutputExpression)


Input Expr: Mult(Const(50.0),Ident(x))
Output Expr: Plus(Mult(Const(0.0),Ident(x)),Mult(Const(1.0),Const(50.0)))


InputExpression: Mult = Mult(e1 = Const(v = 50.0), e2 = Ident(x = "x"))
OutputExpression: Expr = Plus(
  e1 = Mult(e1 = Const(v = 0.0), e2 = Ident(x = "x")),
  e2 = Mult(e1 = Const(v = 1.0), e2 = Const(v = 50.0))
)

### That's all folks!